In [ ]:
import numpy as np
import math
import time

MAXLINES = 100000
NVALUES = 5
PI = 3.1415926535897932385

# Constants
KAPPALO = 500.0
KAPPAHI = 750.0
DELTAKAPPA = 0.0603 / 2

# Initialize data structure to store VAL
VAL = np.zeros((MAXLINES, NVALUES))

# Read HiTran data from the input file
input_file = r'HiTranCO2.txt'
output_file = r'CO2abs.csv'

nl = 0

try:
    with open(input_file, 'r') as file:
        for line in file:
            # Formatting in Python is different, we read from the file directly
            if nl >= MAXLINES:
                break
            elements = line.split()
            if len(elements) < 7:
                continue
            ia = int(elements[0])  # Unused
            ib = int(elements[1])  # Unused
            for j in range(NVALUES):
                VAL[nl, j] = float(elements[j + 2])
            nl += 1
except FileNotFoundError:
    print(f"File {input_file} not found.")
    exit(1)

NLINES = nl

# Function to calculate the indefinite integral of Lorentz line shape function
def ILNZ(k, k0, g):
    return math.atan2((k - k0), g) / PI

# Perform integration and write to the output file
start_time = time.time()

with open(output_file, 'w') as file:
    for kappa in np.arange(KAPPALO + DELTAKAPPA, KAPPAHI + DELTAKAPPA, 2 * DELTAKAPPA):
        S = 0.0
        for line in range(NLINES):
            v1 = ILNZ(kappa + DELTAKAPPA, VAL[line, 0], VAL[line, 3])
            v2 = ILNZ(kappa - DELTAKAPPA, VAL[line, 0], VAL[line, 3])
            V = v1 - v2
            S += VAL[line, 1] * V
        S = S / (2 * DELTAKAPPA)
        file.write(f"{kappa:.6E},{S:.6E}\n")
        # Optional console output
        print(f"{kappa:.6E},{S:.6E}")

end_time = time.time()

# Optional statistics output
print(f"{NLINES} Lines, {(KAPPAHI - KAPPALO) / 2 / DELTAKAPPA} Values, {end_time - start_time} seconds")
